### 0. Utilities

#### Package imports

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

from utils.industry import IndustryStandard
from utils.inference import (export_inference_to_csv, insert_levels,
                             insert_parents, load_raw_inference)
from utils.inference.industry import (INDUSTRY_INFERENCE_PATHS,
                                      RAW_INDUSTRY_INFERENCE_PATHS,
                                      default_clean)
from utils.inference.resource import (RAW_RESOURCE_INFERENCE_PATHS,
                                      RESOURCE_INFERENCE_PATHS)
from utils.resource import ResourceStandard

### 1. Text cleaning

- Manually cleared the print area to remove warnings produced after reading from spreadsheets

In [2]:
industry_dfs = load_raw_inference(RAW_INDUSTRY_INFERENCE_PATHS)

In [3]:
resource_dfs = load_raw_inference(RAW_RESOURCE_INFERENCE_PATHS)

#### Industry classification standards

##### Cleaning ISIC data

In [4]:
# Read the CSV file
ISIC_df = industry_dfs[IndustryStandard.ISIC][0]

# Set the 'Code' column as the index
ISIC_df = ISIC_df.set_index("Code")

# Add a 'Level' column
insert_levels(ISIC_df)

# Add a 'Parent' column
insert_parents(ISIC_df)

# Extra cleaning techniques
default_clean(ISIC_df)

industry_dfs[IndustryStandard.ISIC] = ISIC_df

In [5]:
ISIC_df.head()

,Level,Parent,Description
Code,,,
A,1,,"Agriculture, forestry and fishing"
01,2,A,"Crop and animal production, hunting and relate..."
011,3,01,Growing of non-perennial crops
0111,4,011,"Growing of cereals (except rice), leguminous c..."
0112,4,011,Growing of rice


##### Cleaning NACE data

In [6]:
# Read the Excel file
NACE_df = industry_dfs[IndustryStandard.NACE][0]

# Replace NaN values with empty strings
NACE_df = NACE_df.fillna("")

# Set the 'Code' column as the index
NACE_df = NACE_df.set_index("Code")

# Rename columns
NACE_df = NACE_df.rename(columns={
    "This item includes": "Examples",
    "This item excludes": "Exclusions",
    "Reference to ISIC Rev. 4": "ISIC code"
})

# Concatenate the columns "This item also includes" and "Definition"
NACE_df["Examples"] += " " + NACE_df["This item also includes"]

# Drop columns
NACE_df = NACE_df.drop(columns=["Order", "Rulings", "This item also includes"])

# Extra cleaning techniques
default_clean(NACE_df)

industry_dfs[IndustryStandard.NACE] = NACE_df

In [7]:
NACE_df.head()

,Level,Parent,Description,Examples,Exclusions,ISIC code
Code,,,,,,
A,1,,"AGRICULTURE, FORESTRY AND FISHING",This section includes the exploitation of vege...,,A
01,2,A,"Crop and animal production, hunting and relate...","This division includes two basic activities, n...",Agricultural activities exclude any subsequent...,01
01.1,3,01,Growing of non-perennial crops,This group includes the growing of non-perenni...,,011
01.11,4,01.1,"Growing of cereals (except rice), leguminous c...",This class includes all forms of growing of ce...,"This class excludes:\n- growing of rice, see 0...",0111
01.12,4,01.1,Growing of rice,This class includes:\n- growing of rice (inclu...,,0112


##### Cleaning WZ data

In [8]:
# Read the Excel file
WZ_df = industry_dfs[IndustryStandard.WZ][0]

# Rename columns
WZ_df = WZ_df.rename(columns={"Code WZ 2008": "Code", "Title": "Description"})

# Set the 'Code' column as the index
WZ_df = WZ_df.set_index("Code")

# Convert the 'Level' column to integers
WZ_df["Level"] = WZ_df["Level"].astype(int)

# Add a 'Parent' column
insert_parents(WZ_df)

# Remove columns
WZ_df = WZ_df.drop(columns="Unit(s) of measure")

# Extra cleaning techniques
default_clean(WZ_df)

industry_dfs[IndustryStandard.WZ] = WZ_df

In [9]:
WZ_df.head()

,Level,Parent,Description
Code,,,
A,1,,"Agriculture, forestry and fishing"
01,2,A,"Crop and animal production, hunting and relate..."
01.1,3,01,Growing of non-perennial crops
01.11,4,01.1,"Growing of cereals (except rice), leguminous c..."
01.11.0,5,01.11,"Growing of cereals (except rice), leguminous c..."


##### Cleaning SSIC v1 data

In [10]:
# Read the Excel file
SSIC_v1_df = industry_dfs[IndustryStandard.SSIC][0]

# Drop columns
SSIC_v1_df = SSIC_v1_df.drop(columns=["ISIC Rev. 4 Part", "ISIC Rev. 4 Title"])

# Rename columns
SSIC_v1_df = SSIC_v1_df.rename(columns={
    "SSIC 2020 Title": "Description",
    "SSIC 2020": "Code",
    "ISIC Rev. 4": "ISIC code"
})

# Set the 'Code' column as the index
SSIC_v1_df = SSIC_v1_df.set_index("Code")

# Add a 'Level' column
insert_levels(SSIC_v1_df)

# Extra cleaning techniques
default_clean(SSIC_v1_df)

In [11]:
SSIC_v1_df.head()

,Level,Description,ISIC code
Code,,,
01111,5,Growing of leafy and fruit vegetables,0113
01112,5,Growing of mushrooms,0113
01113,5,Growing of root crops,0113
01119,5,Growing of food crops (non-hydroponics) n.e.c.,0111
01119,5,Growing of food crops (non-hydroponics) n.e.c.,0112


##### Cleaning SSIC v2 data

In [12]:
# Read the Excel file
SSIC_v2_df = industry_dfs[IndustryStandard.SSIC][1]

# Drop columns
SSIC_v2_df = SSIC_v2_df.drop(columns=["Cross References", "Groups Classified Under this Code"])

# Replace '<Blank>' with empty strings
SSIC_v2_df = SSIC_v2_df.replace(to_replace="<Blank>", value="")

# Rename columns
SSIC_v2_df = SSIC_v2_df.rename(columns={
    "SSIC 2020 Title": "Description",
    "SSIC 2020": "Code",
    "Detailed Definitions": "Definition",
    "Examples of Activities Classified Under this Code": "Examples"
})

# Set the 'Code' column as the index
SSIC_v2_df = SSIC_v2_df.set_index("Code")

# Add a 'Level' column
insert_levels(SSIC_v2_df)

# Add a 'Parent' column
insert_parents(SSIC_v2_df)

# Extra cleaning techniques
default_clean(SSIC_v2_df)

industry_dfs[IndustryStandard.SSIC] = SSIC_v2_df

In [13]:
SSIC_v2_df.head()

,Level,Parent,Description,Definition,Examples
Code,,,,,
A,1,,AGRICULTURE AND FISHING,,
01,2,A,AGRICULTURE AND RELATED SERVICE ACTIVITIES,,
011,3,01,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",,
0111,4,011,Growing of Food Crops (Non-Hydroponics),,
01111,5,0111,Growing of leafy and fruit vegetables,This sub-class includes the cultivation of lea...,


#### Resource classification standards

##### Cleaning CPC data

In [14]:
# Read the CSV file
CPC_df = resource_dfs[ResourceStandard.CPC][0]

# Rename columns
CPC_df = CPC_df.rename(columns={"code": "Code", "name": "Description"})

# Moving the first occurring codes 1-4 and their children under code 0 by prefixing them with '0'

# Indexing Pandas data frames: integer rows, named columns
# Answer source: https://stackoverflow.com/a/59275697
CPC_df.iloc[1:417, CPC_df.columns.get_loc("Code")] = CPC_df.iloc[1:417, CPC_df.columns.get_loc("Code")].map(lambda x: f"0{x}")

# Set the 'Code' column as the index
CPC_df = CPC_df.set_index("Code")

# Add a 'Level' column
insert_levels(CPC_df)

# Add a 'Parent' column
insert_parents(CPC_df)

resource_dfs[ResourceStandard.CPC] = CPC_df

In [15]:
CPC_df.head()

,Level,Parent,Description
Code,,,
0,1,,"Agriculture, forestry and fishery products"
01,2,0,"Products of agriculture, horticulture and mark..."
011,3,01,Cereals
0111,4,011,Wheat
01111,5,0111,"Wheat, seed"


### 2. Exporting to CSV

In [16]:
export_inference_to_csv(industry_dfs, INDUSTRY_INFERENCE_PATHS)

In [17]:
export_inference_to_csv(resource_dfs, RESOURCE_INFERENCE_PATHS)